In [6]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 KB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.2/111.2 KB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 KB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 17.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
   

In [7]:
import yfinance as yf
import plotly.graph_objs as go
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots

In [8]:
# The following creates a dataframe of daily market data of SPEU, going back 1 year; by default, the data will collect a year of daily opening prices,
    # daily closing prices, daily price highs, daily price lows, adjusted closing prices, and trading volume
data = yf.download(tickers = 'SPEU', period = '1y', interval = '1d', prepost = True)

[*********************100%***********************]  1 of 1 completed


In [9]:
data.tail() # Displays data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-02-16,37.869999,38.220001,37.820000,37.939999,37.939999,31000
2023-02-17,37.840000,38.200001,37.779999,38.150002,38.150002,24500
2023-02-21,37.910000,38.009998,37.730000,37.779999,37.779999,20900
2023-02-22,37.639999,37.740002,37.470001,37.490002,37.490002,20600
2023-02-23,37.779999,37.769600,37.437000,37.634399,37.634399,20965


In [10]:
# MACD is a technical indicator that tracks the momentum of trends by demonstrating the relationship between two moving averages of the price of a stock
# This calculation, and most calculations, of MACD track the relationship subtract a short (usually 12-period) exponential moving average of the price
    # by a long (usually 26-period) exponential moving average of the price.
# MACD is usually compared against a "signal line" that represents a 9-period exponential moving average of the price, and the intersections between MACD
    # and the signal line indicate a shift from a bear market to a bull market, or vice-versa

ShortEMA = data.Close.ewm(span=12, adjust = False).mean() # Calculates the 12-period EMA
LongEMA = data.Close.ewm(span=26, adjust=False).mean() # Calculates the 26-period EMA
MACD = ShortEMA - LongEMA # Subtracts short EMA by long EMA to arrive at MACD

SignalLine = MACD.ewm(span=9, adjust = False).mean() # Calculates the 9-period EMA to be compared against MACD

In [11]:
data['MACD'] = MACD # adds MACD calculations to SPEU dataframe
data['EMA Signal'] = SignalLine # Adds 9-week EMA to SPEU dataframe
data.tail() # Displays data

,Open,High,Low,Close,Adj Close,Volume,MACD,EMA Signal
Date,,,,,,,,
2023-02-16,37.869999,38.220001,37.820000,37.939999,37.939999,31000,0.386168,0.483392
2023-02-17,37.840000,38.200001,37.779999,38.150002,38.150002,24500,0.373119,0.461338
2023-02-21,37.910000,38.009998,37.730000,37.779999,37.779999,20900,0.329127,0.434896
2023-02-22,37.639999,37.740002,37.470001,37.490002,37.490002,20600,0.267777,0.401472
2023-02-23,37.779999,37.769600,37.437000,37.634399,37.634399,20965,0.228177,0.366813


In [12]:
# ADX is a technical indicator that tracks the strength of a market trend. Note that it indicates only the magnitude of the trend, and not the direction;
    # thus, a higher ADX indicates only the presence of either a bearish or bullish market.


high = data['High'] # Sets high equal to the daily price peaks of SPEU
low = data['Low'] # Sets low equal to the daily price troughs of SPEU
close = data['Close'] # Sets close equal to the daily closing prices of SPEU
lookback = 14 # Sets the "lookback" period equal to 14 periods, which is standard

plus_dm = high.diff() # Establishes the positive Directional Movement (+DM) of SPEU using price peaks
minus_dm = low.diff() # Establishes the negative Directional Movement (-DM) of SPEU using price troughs
plus_dm[plus_dm < 0] = 0 # Normalizes positive Directional Movement to only be positive
minus_dm[minus_dm > 0] = 0 # Normalizes negative Directional Movement to only be negative

# Creates 3 dataframes to measure three differences
tr1 = pd.DataFrame(high - low) # Calculates difference between a day's price peak and the same day's price trough
tr2 = pd.DataFrame(abs(high - close.shift(1))) # Calculates absolute difference between day's price high and next day's closing price
tr3 = pd.DataFrame(abs(low - close.shift(1))) # Calculates absolute difference between day's price low and next day's closing price

frames = [tr1, tr2, tr3] # Merges the 3 differences into one dataframe (3 columns)

# Calculates the true range (TR) by selecting the maximum of the three differences for each index(day)
tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1) # consolidates the dataframe into one column representing the TR (max of differences)

# Average true range (ATR) is calculated by taking the average true range of the lookback period (14 periods)
atr = tr.rolling(lookback).mean()


plus_di = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr) # Establishes positive Directional Index(+DI) = EMA of +DM / ATR
minus_di = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr)) # Establishes negative Directional Index(-DI) = EMA of -DM / ATR

dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100 # Calculates Directional Index(DI) using +DI and -DI 
adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback # Calculates the Average Directional Index(ADX) using DI and the lookback period
adx_smooth = adx.ewm(alpha = 1/lookback).mean() # Smooths ADX to provide more accurate values by using a custom moving average

In [13]:
data['Avg Directional Index'] = adx_smooth # Adds (smoothed) ADX calculations to SPEU dataframe
data.tail() # Displays data

,Open,High,Low,Close,Adj Close,Volume,MACD,EMA Signal,Avg Directional Index
Date,,,,,,,,,
2023-02-16,37.869999,38.220001,37.820000,37.939999,37.939999,31000,0.386168,0.483392,31.396168
2023-02-17,37.840000,38.200001,37.779999,38.150002,38.150002,24500,0.373119,0.461338,31.099362
2023-02-21,37.910000,38.009998,37.730000,37.779999,37.779999,20900,0.329127,0.434896,30.711332
2023-02-22,37.639999,37.740002,37.470001,37.490002,37.490002,20600,0.267777,0.401472,30.165371
2023-02-23,37.779999,37.769600,37.437000,37.634399,37.634399,20965,0.228177,0.366813,28.980508


In [15]:
# Visualizes data from two indicators with respect to SPEU

# Creates 2 separate plots (since MACD values don't visually scale well with ADX values) - a MACD plot and an ADX plot with a shared x-axis(time)
SPEUfig = make_subplots(rows = 2, cols =1, shared_xaxes = True, subplot_titles = ("SPEU Live Moving Average Convergence/Divergence", "SPEU Live Average Directional Movement Index"))

# Establishes MACD and EMASignal series for upper plot
SPEUfig.append_trace(go.Scatter(x=data.index, y = MACD, line=dict(color='blue', width = .8), name = 'MACD'), row =1, col = 1)
SPEUfig.append_trace(go.Scatter(x=data.index, y = SignalLine, line = dict(color='red', width = .8), name = '9-wk EMA Signal Line'), row = 1, col = 1)

# Establishes ADX series for lower plot
SPEUfig.append_trace(go.Scatter(x=data.index, y = adx_smooth, line = dict(color='green', width = .8), name = "Average Directional Index"), row = 2, col = 1)

# Allows viewer to dynamically adjust the time interval for the indicator of interest; applies to both plots simultaneously
SPEUfig.update_xaxes(
    rangeslider_visible=False,
    rangeselector=dict(
        buttons=list([
            dict(count=15, label="1 Week", step="day", stepmode="backward"),
            dict(count=45, label="1 Month", step="day", stepmode="backward"),
            dict(count=1, label="HTD", step="hour", stepmode="todate"),
            dict(count=3, label="1 Day", step="day", stepmode="backward"),
            dict(step="all")
        ])
    )
)

# Sets axis titles for each plot
SPEUfig['layout']['xaxis']['title'] = 'Date'
SPEUfig['layout']['xaxis2']['title']= 'Date'
SPEUfig['layout']['yaxis']['title'] = 'MACD Value'
SPEUfig['layout']['yaxis2']['title'] = 'ADX Value'

# Sets title for the visual containing both plots
SPEUfig.update_layout(title_text='SPDR Portfolio Europe ETF: Live MACD and ADX Performance', xaxis_rangeslider_visible = False, height = 600)

# Displays the visual containing both plots
SPEUfig.show()